In [ ]:
!pip install bs4

In [ ]:
import requests
import time
from bs4 import BeautifulSoup

In [ ]:
url = "https://books.toscrape.com/"
response = requests.get(url)
scraped = BeautifulSoup(response.text, 'html.parser')

In [ ]:
scraped

In [ ]:
books = scraped.find_all("h3")
books

start_time = time.time()
books_extracted = 0

for book in books:
  book_url = book.find('a')['href']
  book_res = requests.get(url+book_url)
  book_scraped = BeautifulSoup(book_res.content, "html.parser")

  title = book_scraped.find('h1').text
  category = book_scraped.find('ul', class_="breadcrumb").find_all("a")[2].text.strip()
  rating = book_scraped.find('p', class_="star-rating")['class'][1]
  price = book_scraped.find('p', class_="price_color").text.strip()
  avaialability = book_scraped.find('p', class_="availability").text.strip()

  books_extracted += 1

  end_time = time.time()
  tot_time = (end_time - start_time)/60.0
  print(f'{title} \n',
        f'{category} \n',
        f'{rating} \n',
        f'{price} \n',
        f'{avaialability}'
  )

In [ ]:
books_data = []

for page_num in range(1, 51):
  url = f'https://books.toscrape.com/catalogue/page-{page_num}.html'
  response = requests.get(url)
  scraped = BeautifulSoup(response.text, 'html.parser')

  books = scraped.find_all("h3")

  start_time = time.time()

  for book in books:
    book_url = book.find('a')['href']
    book_res = requests.get('https://books.toscrape.com/catalogue/'+book_url)
    book_scraped = BeautifulSoup(book_res.content, "html.parser")

    title = book_scraped.find('h1').text
    category = book_scraped.find('ul', class_="breadcrumb").find_all("a")[2].text.strip()
    rating = book_scraped.find('p', class_="star-rating")['class'][1]
    price = book_scraped.find('p', class_="price_color").text.strip()
    avaialability = book_scraped.find('p', class_="availability").text.strip()

    books_data.append([title, category, rating, price, avaialability])
    books_extracted += 1
    end_time = time.time()
    tot_time = (end_time - start_time)/60.0
    print(title, category, rating, price, avaialability)
    print(books_data)

    print(f'{page_num * len(books)} books extracted so far ....')


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(books_data, columns=["Title", "Category","Rating","Price","Avaialability"])

In [ ]:
df.head(20)

In [ ]:
df.to_csv("books_scraped.csv", index=False)
print("Books scraped have been saved to csv file")